https://huggingface.co/docs/transformers/en/tasks/token_classification

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
os.environ['AUTOGRAPH_VERBOSITY'] = '1'
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

#!pip3 install -qU tensorflow tf_keras transformers datasets evaluate seqeval sentencepiece

In [2]:
#from huggingface_hub import notebook_login
#notebook_login()

In [3]:
import tensorflow as tf

from datasets import load_dataset
from transformers import (
    create_optimizer,
    pipeline,
    AutoTokenizer,
    BigBirdTokenizerFast,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TFAutoModelForTokenClassification,
    TrainingArguments,
    Trainer
)

from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback

In [4]:
wnut = load_dataset("wnut_17")
wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [5]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [6]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [7]:
import evaluate
import numpy as np

seqeval = evaluate.load("seqeval")

labels = [label_list[i] for i in wnut["train"][0][f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

def tokenize_and_align_labels(examples): # TODO: PASS TOKENIXER
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
if True:
    # PyTorch
    model_path = 'google/bigbird-roberta-base'
    model_checkpoint = 'bigbird-roberta-ner' # marksusol/bigbird-roberta-ner
    model_output_dir = 'bigbird-roberta-ner'
    tokenizer = BigBirdTokenizerFast.from_pretrained(model_path)
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, max_length=4096)
    model = AutoModelForTokenClassification.from_pretrained(model_path, num_labels=13, id2label=id2label, label2id=label2id)

    tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

    training_args = TrainingArguments(
        output_dir=model_output_dir,
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        dataloader_num_workers=2,
        #dataloader_prefetch_factor=1,
        #num_workers=1,  # enable multiprocessing.
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        # push_to_hub=True, 
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_wnut["train"],
        eval_dataset=tokenized_wnut["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    tokenizer.save_pretrained(model_checkpoint)
    model.save_pretrained(model_checkpoint)
else:
    # TensorFlow
    model_path = 'distilbert/distilbert-base-uncased'
    model_checkpoint = 'distilbert-ner' # marksusol/distilbert-ner
    model_output_dir = 'distilbert-ner'
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

    model = TFAutoModelForTokenClassification.from_pretrained(model_path, num_labels=13, id2label=id2label, label2id=label2id)

    tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

    tf_train_set = model.prepare_tf_dataset(
        tokenized_wnut["train"],
        shuffle=True,
        batch_size=16,
        collate_fn=data_collator,
    )

    tf_validation_set = model.prepare_tf_dataset(
        tokenized_wnut["validation"],
        shuffle=False,
        batch_size=16,
        collate_fn=data_collator,
    )

    batch_size = 16
    num_train_epochs = 3
    num_train_steps = (len(tokenized_wnut["train"]) // batch_size) * num_train_epochs
    optimizer, lr_schedule = create_optimizer(
        init_lr=2e-5,
        num_train_steps=num_train_steps,
        weight_decay_rate=0.01,
        num_warmup_steps=0,
    )
    
    model.compile(optimizer=optimizer)  # No loss argument!
    metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

    #push_to_hub_callback = PushToHubCallback(
    #    output_dir=model_output_dir,
    #    tokenizer=tokenizer,
    #)

    callbacks = [metric_callback] #, push_to_hub_callback]
    model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

    tokenizer.save_pretrained(model_checkpoint)
    model.save_pretrained(model_checkpoint)


Some weights of BigBirdForTokenClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Python(2805) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(2806) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(2807) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(2815) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(2816) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb: Currently logged in as: marksusol. Use `wandb login --relogin` to force relogin
Python(2825) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(2832) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(2838) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.335738,0.275862,0.066728,0.107463,0.927973
2,No log,0.318458,0.351206,0.121409,0.180441,0.930709


Python(3021) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(3026) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad

## Inference

In [12]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

#classifier = pipeline("ner", model="marksusol/bigbird-roberta-ner")
classifier = pipeline("ner", model=model_checkpoint)
classifier(text)

Attention type 'block_sparse' is not possible if sequence_length: 17 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


[{'entity': 'B-group',
  'score': 0.12964632,
  'index': 1,
  'word': '▁The',
  'start': 0,
  'end': 3},
 {'entity': 'B-group',
  'score': 0.11717238,
  'index': 2,
  'word': '▁Golden',
  'start': 3,
  'end': 10},
 {'entity': 'B-location',
  'score': 0.13189259,
  'index': 13,
  'word': '▁San',
  'start': 79,
  'end': 83}]

In [10]:
# what the pipeline does above
if False:
    from transformers import AutoTokenizer
    from transformers import TFAutoModelForTokenClassification
    
    #tokenizer = AutoTokenizer.from_pretrained("marksusol/bigbird-roberta-ner")
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    inputs = tokenizer(text, return_tensors="tf")
    
    #model = TFAutoModelForTokenClassification.from_pretrained("marksusol/bigbird-roberta-ner")
    model = TFAutoModelForTokenClassification.from_pretrained(model_checkpoint)
    logits = model(**inputs).logits
    
    predicted_token_class_ids = tf.math.argmax(logits, axis=-1)
    predicted_token_class = [model.config.id2label[t] for t in predicted_token_class_ids[0].numpy().tolist()]
    predicted_token_class